# Prediction per ticker

In this notebook, an action recommendation for the stock days in the first half of 2018 is predicted for each individual ticker. The individual steps from the first iteration are performed one after the other. First, the internal data are processed with time histories of 10, 20, 30, 40, 50, 60, 70, 80 and 90 days. Then the training and testing with random forest will be carried out, as random forest is the method with the best results according to the first iteration. Then the prediction for the stock days in 2018 is made, followed by these steps for the next ticker and so on. 

# Content
 1. Import dependencies
 2. Specification of trading days
 3. Helpers
 4. Data preperation, training, testing and prediction per ticker
   1. Load data
   2. Prepare data frame for price
   3. Prepare data frame for volume
   4. Merging data frames
   5. Add action recommondations
   6. Splitting Data in training and testing
   7. Random Forest
   8. Pediction for trading days 2018
 5. Saving results
 6. Result

<hr>

# 1. Import dependencies

In [1]:
from os import listdir
from os.path import isfile, join, normpath
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series
from sklearn import preprocessing
from pandas import concat
import datetime
from datetime import timedelta, date
from sklearn.model_selection import train_test_split, GridSearchCV,  cross_val_score
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import csv 

# 2. Specification of trading days

In order for the following method to work, the share days must be available for the trading days in the first half of 2018. In addition, the last 90 trading days of 2017 must be available, since time histories of up to 90 days are used retroactively.

In [2]:
boersen_days_2018 = [
    "2018-01-02",
    "2018-01-03",
    "2018-01-04", 
    "2018-01-05",
    "2018-01-08",
    "2018-01-09",
    "2018-01-10",
    "2018-01-11",
    "2018-01-12",
    "2018-01-16",
    "2018-01-17",
    "2018-01-18",
    "2018-01-19",
    "2018-01-22",
    "2018-01-23",
    "2018-01-24",
    "2018-01-25",
    "2018-01-26",
    "2018-01-29",
    "2018-01-30",
    "2018-01-31",
    "2018-02-01",
    "2018-02-02",
    "2018-02-05",
    "2018-02-06",
    "2018-02-07",
    "2018-02-08",
    "2018-02-09",
    "2018-02-12",
    "2018-02-13",
    "2018-02-14",
    "2018-02-15",
    "2018-02-16",
    "2018-02-20",
    "2018-02-21",
    "2018-02-22",
    "2018-02-23",
    "2018-02-26",
    "2018-02-27",
    "2018-02-28",
    "2018-03-01",
    "2018-03-02",
    "2018-03-05",
    "2018-03-06",
    "2018-03-07",
    "2018-03-08",
    "2018-03-09",
    "2018-03-12",
    "2018-03-13",
    "2018-03-14",
    "2018-03-15",
    "2018-03-16",
    "2018-03-19",
    "2018-03-20",
    "2018-03-21",
    "2018-03-22",
    "2018-03-23",
    "2018-03-26",
    "2018-03-27",
    "2018-03-28",
    "2018-03-29",
    "2018-04-02",
    "2018-04-03",
    "2018-04-04",
    "2018-04-05",
    "2018-04-06",
    "2018-04-09",
    "2018-04-10",
    "2018-04-11",
    "2018-04-12",
    "2018-04-13",
    "2018-04-16",
    "2018-04-17",
    "2018-04-18",
    "2018-04-19",
    "2018-04-20",
    "2018-04-23",
    "2018-04-24",
    "2018-04-25",
    "2018-04-26",
    "2018-04-27",
    "2018-04-30",
    "2018-05-01",
    "2018-05-02",
    "2018-05-03",
    "2018-05-04",
    "2018-05-07",
    "2018-05-08",
    "2018-05-09",
    "2018-05-10",
    "2018-05-11",
    "2018-05-14",
    "2018-05-15",
    "2018-05-16",
    "2018-05-17",
    "2018-05-18",
    "2018-05-21",
    "2018-05-22",
    "2018-05-23",
    "2018-05-24",
    "2018-05-25",
    "2018-05-29",
    "2018-05-30",
    "2018-05-31",
    "2018-06-01",
    "2018-06-04",
    "2018-06-05",
    "2018-06-06",
    "2018-06-07",
    "2018-06-08",
    "2018-06-11",
    "2018-06-12",
    "2018-06-13",
    "2018-06-14",
    "2018-06-15",
    "2018-06-18",
    "2018-06-19",
    "2018-06-20",
    "2018-06-21",
    "2018-06-22",
    "2018-06-25",
    "2018-06-26",
    "2018-06-27",
    "2018-06-28",
    "2018-06-29"
]

In [3]:
boersen_days_2017 = [
    "2017-08-23",
    "2017-08-24",
    "2017-08-25",
    "2017-08-28",
    "2017-08-29",
    "2017-08-30",
    "2017-08-31",
    "2017-09-01",
    "2017-09-05",
    "2017-09-06",
    "2017-09-07",
    "2017-09-08",
    "2017-09-11",
    "2017-09-12",
    "2017-09-13",
    "2017-09-14",
    "2017-09-15",
    "2017-09-18",
    "2017-09-19",
    "2017-09-20",
    "2017-09-21",
    "2017-09-22",
    "2017-09-25",
    "2017-09-26",
    "2017-09-27",
    "2017-09-28",
    "2017-09-29",
    "2017-10-02",
    "2017-10-03",
    "2017-10-04",
    "2017-10-05",
    "2017-10-06",
    "2017-10-09",
    "2017-10-10",
    "2017-10-11",
    "2017-10-12",
    "2017-10-13",
    "2017-10-16",
    "2017-10-17",
    "2017-10-18",
    "2017-10-19",
    "2017-10-20",
    "2017-10-23",
    "2017-10-24",
    "2017-10-25",
    "2017-10-26",
    "2017-10-27",
    "2017-10-30",
    "2017-10-31",
    "2017-11-01",
    "2017-11-02",
    "2017-11-03",
    "2017-11-06",
    "2017-11-07",
    "2017-11-08",
    "2017-11-09",
    "2017-11-10",
    "2017-11-13",
    "2017-11-14",
    "2017-11-15",
    "2017-11-16",
    "2017-11-17",
    "2017-11-20",
    "2017-11-21",
    "2017-11-22",
    "2017-11-24",
    "2017-11-27",
    "2017-11-28",
    "2017-11-29",
    "2017-11-30",
    "2017-12-01",
    "2017-12-04",
    "2017-12-05",
    "2017-12-06",
    "2017-12-07",
    "2017-12-08",
    "2017-12-11",
    "2017-12-12",
    "2017-12-13",
    "2017-12-14",
    "2017-12-15",
    "2017-12-18",
    "2017-12-19",
    "2017-12-20",
    "2017-12-21",
    "2017-12-22",
    "2017-12-26",
    "2017-12-27",
    "2017-12-28",
    "2017-12-29"
]

In [4]:
# For the training and testing at least the above mentioned trading days in 2017 are needed.For prediction, 
# the above-mentioned trading days must be available in 2018.
all_boersen_days = boersen_days_2017
all_boersen_days.extend(boersen_days_2018)

# 3. Helpers

In [5]:
# fucntion for creating of time coureses
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [6]:
# List on which the results of Kaggle are written
result_str_lst = list()

In [7]:
# definition of path where files can be found
dir_path = "C:/Users/Katja/Desktop/Big Data/Projekt/data/"
dir_path = normpath(dir_path)

In [8]:
# read tickers file names
ticker_path = join(dir_path, "stocks")
ticker_file_names = [f for f in listdir(ticker_path) if (isfile(join(ticker_path, f)) and ".csv" in f)]

# 4. Data preperation, training, testing and prediction per ticker

In [9]:
for file_name in ticker_file_names:

    ticker_name = file_name.replace(".csv", "")

    
    # A Load data
    file_str = join(ticker_path, file_name)
    df_ticker = pd.read_csv(file_str)
    df_ticker.columns = ['Date', 'Open', 'Close', 'Low', 'High', 'Volume']

    # Checking whether required trading days are available. If the data is complete, 
    # the preparation of the data is started.
    ticker_data_valid = True
    for date_str in all_boersen_days:
        if not date_str in df_ticker['Date'].values:
            ticker_data_valid = False
            continue

    # If the data for a ticker is not completely available, a "0" should be predicted 
    # for each trading day on the result list. In addition, a message should be displayed 
    # that the data of the ticker is incomplete.
    if not ticker_data_valid:
        for date_str in boersen_days_2018:
            result_str_lst.append([date_str + ":" + ticker_name, 0])
        print(ticker_name + " not complete")
        continue

        
    # B Prepare data frame for price
    # Mid-prices consists of the average of the values low and high. 
    df_ticker['Mid_prices'] = (df_ticker.Low + df_ticker.High) / 2
    df_for_midprices = df_ticker.copy(deep=True)
    # Apart from the new Mid-prices column, no other values are required in this step.
    df_midprices = df_for_midprices.drop(
        ['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], axis=1)

    # For prices, time histories of 10, 20, 30, 40, 50, 60, 70, 80 and 90 are calculated 
    # before the respective trading day.
    midprices = df_midprices.values
    reframed_midprices = series_to_supervised(midprices, 0, 91)

    entwicklungsrate_midprices_10 = (
        1/reframed_midprices['var1(t)']*reframed_midprices['var1(t+10)'])-1
    entwicklungsrate_midprices_20 = (
        1/reframed_midprices['var1(t)']*reframed_midprices['var1(t+20)'])-1
    entwicklungsrate_midprices_30 = (
        1/reframed_midprices['var1(t)']*reframed_midprices['var1(t+30)'])-1
    entwicklungsrate_midprices_40 = (
        1/reframed_midprices['var1(t)']*reframed_midprices['var1(t+40)'])-1
    entwicklungsrate_midprices_50 = (
        1/reframed_midprices['var1(t)']*reframed_midprices['var1(t+50)'])-1
    entwicklungsrate_midprices_60 = (
        1/reframed_midprices['var1(t)']*reframed_midprices['var1(t+60)'])-1
    entwicklungsrate_midprices_70 = (
        1/reframed_midprices['var1(t)']*reframed_midprices['var1(t+70)'])-1
    entwicklungsrate_midprices_80 = (
        1/reframed_midprices['var1(t)']*reframed_midprices['var1(t+80)'])-1
    entwicklungsrate_midprices_90 = (
        1/reframed_midprices['var1(t)']*reframed_midprices['var1(t+90)'])-1

    # Renaming the column headings and recording the time coureses in the dataframe.
    df_for_midprices['Entwicklungsrate Preis t+10'] = entwicklungsrate_midprices_10
    df_for_midprices['Entwicklungsrate Preis t+20'] = entwicklungsrate_midprices_20
    df_for_midprices['Entwicklungsrate Preis t+30'] = entwicklungsrate_midprices_30
    df_for_midprices['Entwicklungsrate Preis t+40'] = entwicklungsrate_midprices_40
    df_for_midprices['Entwicklungsrate Preis t+50'] = entwicklungsrate_midprices_50
    df_for_midprices['Entwicklungsrate Preis t+60'] = entwicklungsrate_midprices_60
    df_for_midprices['Entwicklungsrate Preis t+70'] = entwicklungsrate_midprices_70
    df_for_midprices['Entwicklungsrate Preis t+80'] = entwicklungsrate_midprices_80
    df_for_midprices['Entwicklungsrate Preis t+90'] = entwicklungsrate_midprices_90
    df_for_midprices = df_for_midprices.drop(
        ['Open', 'High', 'Low', 'Close', 'Volume', 'Mid_prices'], axis=1)

    
    # C Prepare data frame for volume
    # The same calculation is performed for the volume as for midprices.
    df_for_volumes = df_ticker.copy(deep=True)
    df_volumes = df_for_volumes.drop(
        ['Date', 'Open', 'High', 'Low', 'Close', 'Mid_prices'], axis=1)

    volumes = df_volumes.values
    reframed_volumes = series_to_supervised(volumes, 0, 91)

    entwicklungsrate_volumes_10 = (
        1/reframed_volumes['var1(t)']*reframed_volumes['var1(t+10)'])-1
    entwicklungsrate_volumes_20 = (
        1/reframed_volumes['var1(t)']*reframed_volumes['var1(t+20)'])-1
    entwicklungsrate_volumes_30 = (
        1/reframed_volumes['var1(t)']*reframed_volumes['var1(t+30)'])-1
    entwicklungsrate_volumes_40 = (
        1/reframed_volumes['var1(t)']*reframed_volumes['var1(t+40)'])-1
    entwicklungsrate_volumes_50 = (
        1/reframed_volumes['var1(t)']*reframed_volumes['var1(t+50)'])-1
    entwicklungsrate_volumes_60 = (
        1/reframed_volumes['var1(t)']*reframed_volumes['var1(t+60)'])-1
    entwicklungsrate_volumes_70 = (
        1/reframed_volumes['var1(t)']*reframed_volumes['var1(t+70)'])-1
    entwicklungsrate_volumes_80 = (
        1/reframed_volumes['var1(t)']*reframed_volumes['var1(t+80)'])-1
    entwicklungsrate_volumes_90 = (
        1/reframed_volumes['var1(t)']*reframed_volumes['var1(t+90)'])-1

    df_for_volumes['Entwicklungsrate Volume t+10'] = entwicklungsrate_volumes_10
    df_for_volumes['Entwicklungsrate Volume t+20'] = entwicklungsrate_volumes_20
    df_for_volumes['Entwicklungsrate Volume t+30'] = entwicklungsrate_volumes_30
    df_for_volumes['Entwicklungsrate Volume t+40'] = entwicklungsrate_volumes_40
    df_for_volumes['Entwicklungsrate Volume t+50'] = entwicklungsrate_volumes_50
    df_for_volumes['Entwicklungsrate Volume t+60'] = entwicklungsrate_volumes_60
    df_for_volumes['Entwicklungsrate Volume t+70'] = entwicklungsrate_volumes_70
    df_for_volumes['Entwicklungsrate Volume t+80'] = entwicklungsrate_volumes_80
    df_for_volumes['Entwicklungsrate Volume t+90'] = entwicklungsrate_volumes_90
    df_for_volumes = df_for_volumes.drop(
        ['Open', 'High', 'Low', 'Close', 'Volume', 'Mid_prices'], axis=1)

    
    # D Merging data frames
    # The two data frames for midprices and volumes are merged via the date.
    merged_df = pd.merge(df_for_volumes, df_for_midprices, on='Date')
    
    # Lines containing "NaN" or "inf" are removed from the new data frame. 
    merged_df.dropna(axis = 0, inplace = True)
    merged_df.replace([np.inf, -np.inf], np.nan).dropna(axis=0, inplace=True)
    merged_df = merged_df[~(merged_df['Entwicklungsrate Volume t+10']==np.inf)]


    # E Add action recommondations
    # File with recommendations for action is read in
    df_train_label = pd.read_csv(
        join(dir_path, 'labels_train.csv'), header=0, index_col=0)
    df_train_label = df_train_label.loc[:, df_train_label.columns.intersection([
        ticker_name])]
    df_train_label.columns = ['Y']

    # The recommendations for action and the generated data frame of D are merged by date and sorted by date.
    df_complete = pd.merge(merged_df, df_train_label[['Y']], on='Date')
    df_complete = df_complete.sort_values('Date')
    
    
    # F Splitting Data in training and testing
    x = df_complete[['Entwicklungsrate Preis t+10',
                     'Entwicklungsrate Preis t+20',
                     'Entwicklungsrate Preis t+30',
                     'Entwicklungsrate Preis t+40',
                     'Entwicklungsrate Preis t+50',
                     'Entwicklungsrate Preis t+60',
                     'Entwicklungsrate Preis t+70',
                     'Entwicklungsrate Preis t+80',
                     'Entwicklungsrate Preis t+90',
                     'Entwicklungsrate Volume t+10',
                     'Entwicklungsrate Volume t+20',
                     'Entwicklungsrate Volume t+30',
                     'Entwicklungsrate Volume t+40',
                     'Entwicklungsrate Volume t+50',
                     'Entwicklungsrate Volume t+60',
                     'Entwicklungsrate Volume t+70',
                     'Entwicklungsrate Volume t+80',
                     'Entwicklungsrate Volume t+90']]
    y = df_complete['Y']

    x_train, x_test, y_train, y_test = train_test_split(
        x, y, train_size=0.7, random_state=42)  
    

    # G Random Forest
    rf_class = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_class.fit(x_train, y_train)
    rf_score = rf_class.score(x_test, y_test)
    
    # The score for the accuracy of predictions is output.
    print("Random Forest Score: " + ticker_name + ": " + str(rf_score))
    
    
    # H Pediction for trading days 2018
    selected_dates_df = DataFrame()
    selected_dates = list()
    default_dates = list()

    # The dates to be predicted are determined from the list.
    for date in boersen_days_2018:
        x = merged_df.loc[merged_df['Date'] == date]
        if not x.empty:
            selected_dates_df = selected_dates_df.append(x)
            selected_dates.append(date)
        else:
            default_dates.append(date)

    # For prediction no date is needed and that is why date is dropped.
    selected_dates_df = selected_dates_df.drop(['Date'], axis=1)
    selected_dates_df.head()

    # For each date, the predict function of random Forest is used to give 
    # a recommendation for action.
    prediction = rf_class.predict(selected_dates_df)
    for i in range(len(prediction) + len(default_dates)):
        if len(selected_dates) > 0:
            min_pred = min(selected_dates)
        else:
            min_pred = '9999-99-99'
        if len(default_dates) > 0:
            min_default = min(default_dates)
        else:
            min_default = '9999-99-99'

        # For the given trading days in 2018, a forecast is made as to whether the tickers are 
        # to be held, sold or bought on that day. If a prediction is not possible, the ticker  
        # are to be held and the value 0 is to be predicted.
        if min_pred < min_default:
            min_pred_idx = selected_dates.index(min_pred)
            result_str_lst.append(
                [min_pred + ":" + ticker_name, prediction[min_pred_idx]])
            del selected_dates[min_pred_idx]
            prediction = np.delete(prediction, min_pred_idx)
        else:
            min_default_idx = default_dates.index(min_default)
            result_str_lst.append(
                [min_default + ":" + ticker_name, 0])
            del default_dates[min_default_idx]

C:\Users\Katja\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Random Forest Score: A: 0.9537037037037037


C:\Users\Katja\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Random Forest Score: AAN: 0.9112676056338028


C:\Users\Katja\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


...

Random Forest Score: Y: 0.9682539682539683


C:\Users\Katja\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Random Forest Score: YELP: 0.9138321995464853


C:\Users\Katja\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Random Forest Score: ZBH: 0.966931216931217


# 5. Saving results

In [10]:
# Transfer list to DataFrame
kaggle = pd.DataFrame(data=result_str_lst, columns=['Id', 'Category'])
kaggle.shape

(104375, 2)

In [11]:
# Saving results
kaggle = kaggle.to_csv('Prediction/Random_Forest_prediction_per_ticker.csv', index=False)

# 6. Result

The issued ticker score is between 75.7 % and 99.0 %. Nevertheless, Kaggle only achieves a score of 42.1 %. The training and testing per ticker does not deliver satisfactory results. For example, a learning curve could be analyzed to determine the reasons for this. However, since it is obvious that too little data was used for training and testing, this is not done here. The next step is to combine training and testing for all available tickers. 